In [6]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine

In [7]:
# Import Google Play Store data and Google Play Store User Review into pandas
googleplaystore_csv_filepath = "resources/googleplaystore.csv"
googleplaystore_df = pd.read_csv(googleplaystore_csv_filepath)

googleplaystorereview_csv_filepath = "resources/googleplaystore_user_reviews.csv"
googleplaystorereview_df = pd.read_csv(googleplaystorereview_csv_filepath)

In [8]:
# Transform Google Play Store data
column_names = ["App", "Category", "Rating", "Reviews", "Installs", "Type", "Price", "Genres"]
googleplaystore_transformed = googleplaystore_df[column_names].copy()

googleplaystore_transformed = googleplaystore_transformed.rename(columns = {"App": "app", 
                                                                            "Category": "category", 
                                                                            "Rating": "rating", 
                                                                            "Reviews": "reviews", 
                                                                            "Installs": "installs", 
                                                                            "Type": "type", 
                                                                            "Price": "price", 
                                                                            "Genres": "genre"})


col_name = ["App", "Sentiment", "Sentiment_Polarity", "Sentiment_Subjectivity"]
googleplaystorereview_transformed = googleplaystorereview_df[col_name].copy()

googleplaystorereview_transformed = googleplaystorereview_transformed.rename(columns = {"App": "app",
                                                                                        "Sentiment": "sentiment",
                                                                                        "Sentiment_Polarity": "sentiment polarity score", 
                                                                                        "Sentiment_Subjectivity": "score"})

In [9]:
# Connect to postgres engine
connection_str = "postgres:postgres@localhost:5432/Two_pandas_db"
engine = create_engine(f"postgresql://{connection_str}")

In [10]:
# Load data to postgres database
googleplaystore_transformed.to_sql(name = "google_play_store", con = engine, if_exists = "append", index = False)

googleplaystorereview_transformed.to_sql(name = "google_play_store_review", con = engine, if_exists = "append", index = False)

In [11]:
# View the table names in the database
engine.table_names()

<ipython-input-11-2660315882f6>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['google_play_store', 'google_play_store_review']

In [12]:
# Query google_play_store table
pd.read_sql_query('select * from google_play_store', con=engine).head()


,app,category,rating,reviews,installs,type,price,genre
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,"10,000+",Free,0,Art & Design
1,Coloring book moana,ART_AND_DESIGN,3.9,967,"500,000+",Free,0,Art & Design;Pretend Play
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,"5,000,000+",Free,0,Art & Design
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,"50,000,000+",Free,0,Art & Design
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,"100,000+",Free,0,Art & Design;Creativity


In [13]:
# Query google_play_store_review table
pd.read_sql_query('select * from google_play_store_review', con=engine).head()

,app,sentiment,sentiment polarity score,score
0,10 Best Foods for You,Positive,1.00,0.533333
1,10 Best Foods for You,Positive,0.25,0.288462
2,10 Best Foods for You,None,NaN,NaN
3,10 Best Foods for You,Positive,0.40,0.875000
4,10 Best Foods for You,Positive,1.00,0.300000


In [14]:
# Query join table on specific columns between the two tables
google_master = pd.read_sql_query("""
select gps.app, gps.rating, gps.reviews, gps.price, gps.genre,
gpsr.sentiment, gpsr.score
from google_play_store as gps
join google_play_store_review as gpsr
on gps.app = gpsr.app;
""", con = engine)

In [15]:
# Googleplaystore master join table
google_master.to_sql(name = "google_master", con = engine, if_exists = "append", index = False)